**Ingesting project work breakdown structure from D365 Finance and Operations and extracting columns we need for M7 analysis**

In [22]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql.functions import *

df = spark.read.format("csv").load("Files/Tables/ProfessionalServices/ProjectManagementAndAccounting/Group/ProjPlanVersion/PROJPLANVERSION_00001.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Tables/ProfessionalServices/ProjectManagementAndAccounting/Group/ProjPlanVersion/PROJPLANVERSION_00001.csv".
#display(df)

StatementMeta(, ae0a0d60-ce0d-4fd8-a78f-d0878f0f9cd3, 23, Finished, Available)

Creating a new dataframe that only contains the columns we need from the project work breakdown structure table

In [23]:
renamed_df = df.select(
                col("_c7").alias("ProjectID"),\
                col("_c11").alias("ECDDate"),\
                col("_c5").alias("Group"),\
                col("_c3").alias("ImportDate"),\
                col("_c12").alias("Duration"),\
                col("_c13").alias("EffortInHours"))

StatementMeta(, ae0a0d60-ce0d-4fd8-a78f-d0878f0f9cd3, 24, Finished, Available)

Splitting the project id into project and subproject

In [24]:
split_col = split(renamed_df['ProjectID'],'-')
renamed_df = renamed_df.withColumn('Proj', split_col.getItem(0))
renamed_df = renamed_df.withColumn('SubProj', split_col.getItem(1))

StatementMeta(, ae0a0d60-ce0d-4fd8-a78f-d0878f0f9cd3, 25, Finished, Available)

Sorting by ECD date and rebuilding the dataframe

In [25]:
renamed_df = renamed_df.sort(renamed_df['ECDDate'].desc())

renamed_df = renamed_df.select(
                "Proj",\
                "SubProj",\
                "ECDDate",\
                "Group",\
                "ImportDate",\
                "Duration",\
                "EffortInHours")

#display(renamed_df)

StatementMeta(, ae0a0d60-ce0d-4fd8-a78f-d0878f0f9cd3, 26, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2d8f1d7b-0c7d-45c1-ba3e-e0d8ad3a455b)

Create/overwrite a new table in datalake with the new dataframe

In [26]:
delta_table_path = "Tables/ProjPlanTable" #fill in your delta table path 
renamed_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(delta_table_path)

StatementMeta(, ae0a0d60-ce0d-4fd8-a78f-d0878f0f9cd3, 27, Finished, Available)